<h3>Convert float to hex</h3>

In [1]:
import struct
import binascii
def float_to_hex(f):
    return binascii.hexlify(struct.pack('<f', f))
# float_to_hex(17.5) # 0x418c0000
# print(float_to_hex(17.5).decode("utf-8"))

00008c41


<h3>Reverse order of N (2-bit by 2-bit)</h3>

In [3]:
def reverse_N(N):
    print(N)
    dataLine_hex = str(hex(N))[2:]
    dataLine_hex = dataLine_hex.rjust(8, '0')
    print(dataLine_hex)
    dataLine_hex_reverse = ""
    for i in range(4):
        dataLine_hex_reverse += dataLine_hex [6-2*i: 8-2*i]
    return dataLine_hex_reverse

<h3>10 batches in one packet</h3> 

Each has 799 gradients (50 dataLines)

In [5]:
timeInfo = ["0020", "0000"]
packet_1 = timeInfo 
dataLine_perBatch = 50
for totalBatch in range(10):
    for lineNum in range(dataLine_perBatch):
        inst = ""
        if (lineNum == dataLine_perBatch - 1 and totalBatch == 9):
            inst = inst + '1'
        else:
            inst = inst + '0'
        for floatContent in range(16):
            if(lineNum == 0 and floatContent ==15):
                inst = inst + reverse_N(16*dataLine_perBatch - 1)
            else:
                inst = inst + float_to_hex(totalBatch +lineNum+ floatContent).decode("utf-8")
        if lineNum==0:
            print( "lineNum: ", lineNum, "  -------- batchNum: ", totalBatch, "\n", inst)
        packet_1 = packet_1 + [inst]
        #print(len(inst))
print(len(packet_1))

799
0000031f
lineNum:  0   -------- batchNum:  0 
 0000000000000803f0000004000004040000080400000a0400000c0400000e040000000410000104100002041000030410000404100005041000060411f030000
799
0000031f
lineNum:  0   -------- batchNum:  1 
 00000803f0000004000004040000080400000a0400000c0400000e04000000041000010410000204100003041000040410000504100006041000070411f030000
799
0000031f
lineNum:  0   -------- batchNum:  2 
 00000004000004040000080400000a0400000c0400000e0400000004100001041000020410000304100004041000050410000604100007041000080411f030000
799
0000031f
lineNum:  0   -------- batchNum:  3 
 000004040000080400000a0400000c0400000e040000000410000104100002041000030410000404100005041000060410000704100008041000088411f030000
799
0000031f
lineNum:  0   -------- batchNum:  4 
 0000080400000a0400000c0400000e04000000041000010410000204100003041000040410000504100006041000070410000804100008841000090411f030000
799
0000031f
lineNum:  0   -------- batchNum:  5 
 00000a0400000c0400000e0400000004100001041000

In [6]:
# Result Verification packet_1
for i in range(10):
    total = 0
    for j in range(dataLine_perBatch):
        for k in range(16):
            total += i + j + k 
        #print("Total current: ", i, "Sum of this line currently: ", total - (i + 0 + 15), "----float32: ", float_to_hex(total))
    total = total - (i + 0 + 15)
    print("Total of batch ", i, "is: ",total, "----float32: ", float_to_hex(total)," N is: ", 16*dataLine_perBatch - 1)
    print("Average gradient :"total/(16*dataLine_perBatch-1), "----float32: ", float_to_hex(total/(16*dataLine_perBatch-1)))

Total of batch  0 is:  25585 ----float32:  b'00e2c746'  N is:  799
32.02127659574468 ----float32:  b'ca150042'
Total of batch  1 is:  26384 ----float32:  b'0020ce46'  N is:  799
33.02127659574468 ----float32:  b'ca150442'
Total of batch  2 is:  27183 ----float32:  b'005ed446'  N is:  799
34.02127659574468 ----float32:  b'ca150842'
Total of batch  3 is:  27982 ----float32:  b'009cda46'  N is:  799
35.02127659574468 ----float32:  b'ca150c42'
Total of batch  4 is:  28781 ----float32:  b'00dae046'  N is:  799
36.02127659574468 ----float32:  b'ca151042'
Total of batch  5 is:  29580 ----float32:  b'0018e746'  N is:  799
37.02127659574468 ----float32:  b'ca151442'
Total of batch  6 is:  30379 ----float32:  b'0056ed46'  N is:  799
38.02127659574468 ----float32:  b'ca151842'
Total of batch  7 is:  31178 ----float32:  b'0094f346'  N is:  799
39.02127659574468 ----float32:  b'ca151c42'
Total of batch  8 is:  31977 ----float32:  b'00d2f946'  N is:  799
40.02127659574468 ----float32:  b'ca152042'
T

<h3>10 batches in 3 packets</h3>

Each batch has 479 gradients (30 dataLines), 300 dataLines in total

Each packet has 100 dataLines 

Breaking points: (3 batches + 10 dataLines), (6 batches and 20 datalines), (9 batches and 30 dataline)

In [8]:
timeInfo = ["0050", "0000"]
packet_2_3_4 = timeInfo
dataLine_perBatch = 30
for totalBatch in range(10):
    for lineNum in range(dataLine_perBatch):
        inst = ""
        if((lineNum == 10 and totalBatch == 3) or (lineNum == 20 and totalBatch == 6)): #packet_3 starts
            packet_2_3_4 += timeInfo
        if ((lineNum == 9 and totalBatch == 3) or (lineNum == 19 and totalBatch == 6) or (lineNum == 29 and totalBatch == 9)): #packet_2 or packet_3 end
            inst = inst + '1'   
        else:
            inst = inst + '0'
        for floatContent in range(16):
            if(lineNum == 0 and floatContent ==15):
                inst = inst + reverse_N(16*dataLine_perBatch - 1)
            else:
                inst = inst + float_to_hex(totalBatch +0.5*lineNum+ 0.3*floatContent).decode("utf-8")
        if lineNum==0:
            print( "lineNum: ", lineNum, "  -------- batchNum: ", totalBatch, "\n", inst)
        packet_2_3_4 = packet_2_3_4 + [inst]
print(len(packet_2_3_4))

479
000001df
lineNum:  0   -------- batchNum:  0 
 0000000009a99993e9a99193f6666663f9a99993f0000c03f6666e63f666606409a991940cdcc2c400000404033335340666666409a99794066668640df010000
479
000001df
lineNum:  0   -------- batchNum:  1 
 00000803f6666a63fcdcccc3f3333f33fcdcc0c400000204033333340666646409a995940cdcc6c40000080409a99894033339340cdcc9c406666a640df010000
479
000001df
lineNum:  0   -------- batchNum:  2 
 00000004033331340666626409a993940cdcc4c40000060403333734033338340cdcc8c40666696400000a0409a99a9403333b340cdccbc406666c640df010000
479
000001df
lineNum:  0   -------- batchNum:  3 
 00000404033335340666666409a99794066668640000090409a9999403333a340cdccac406666b6400000c0409a99c9403333d340cdccdc406666e640df010000
479
000001df
lineNum:  0   -------- batchNum:  4 
 0000080409a99894033339340cdcc9c406666a6400000b0409a99b9403333c340cdcccc406666d6400000e0409a99e9403333f340cdccfc4033330341df010000
479
000001df
lineNum:  0   -------- batchNum:  5 
 00000a0409a99a9403333b340cdccbc406666c640000

In [9]:
# Result Verification packet_2_3_4
for i in range(10):
    total = 0
    for j in range(dataLine_perBatch):
        for k in range(16):
            total = total + (i + 0.5*j +0.3*k)
        #print("Batch: ", i, "Sum of this line: ", current, "----float32: ", float_to_hex(current))
    total = total - (i + 0.5 * 0 + 0.3*15)
    print("Total of batch ", i, "is: ",total, "----float32: ", float_to_hex(total)," N is: ", 16*dataLine_perBatch - 1)
    print(total/(16*dataLine_perBatch-1), "----float32: ", float_to_hex(total/(16*dataLine_perBatch-1)))

Total of batch  0 is:  4555.5 ----float32:  b'005c8e45'  N is:  479
9.51043841336117 ----float32:  b'c12a1841'
Total of batch  1 is:  5034.5 ----float32:  b'00549d45'  N is:  479
10.51043841336117 ----float32:  b'c12a2841'
Total of batch  2 is:  5513.5 ----float32:  b'004cac45'  N is:  479
11.51043841336117 ----float32:  b'c12a3841'
Total of batch  3 is:  5992.5 ----float32:  b'0044bb45'  N is:  479
12.51043841336117 ----float32:  b'c12a4841'
Total of batch  4 is:  6471.5 ----float32:  b'003cca45'  N is:  479
13.51043841336117 ----float32:  b'c12a5841'
Total of batch  5 is:  6950.5 ----float32:  b'0034d945'  N is:  479
14.51043841336117 ----float32:  b'c12a6841'
Total of batch  6 is:  7429.5 ----float32:  b'002ce845'  N is:  479
15.51043841336117 ----float32:  b'c12a7841'
Total of batch  7 is:  7908.5 ----float32:  b'0024f745'  N is:  479
16.510438413361168 ----float32:  b'61158441'
Total of batch  8 is:  8387.5 ----float32:  b'000e0346'  N is:  479
17.510438413361168 ----float32:  b'6

<h3>5 batches in 2 packets</h3>
Each batch has 500 gradients

<h3>Write the file</h3>

In [10]:
import os
if os.path.exists("session-data-in.txt"):
    os.remove("session-data-in.txt")
networkPacket = open("session-data-in.txt", "w")
for lineContent in packet_1 + packet_2_3_4:
    networkPacket.writelines(lineContent)
    networkPacket.write("\n")
networkPacket.close()